In [3]:
# Get the time and print it
import datetime

datetime.datetime.now()
print(datetime.datetime.now())

from google.colab import drive
drive.mount('/content/gdrive')

2024-07-29 07:58:43.424022
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
# Olaf de Rohan Willner
# ocd4@cornell.edu / olaf.willner@econ.uzh.ch
# Version: July 27, 2024

# Try installing libraries
import subprocess
import sys
import os

def install_if_not_installed(package_name):
    try:
        __import__(package_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

packages = ["selenium", "webdriver_manager", "datetime", "time", "pyjanitor", "pandas", "fnmatch", "pymssql"]

for package in packages:
    install_if_not_installed(package)

!apt-get update
!apt install firefox

from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from janitor import clean_names
import fnmatch
import datetime
import time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (180 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'htt

In [5]:
# Define webdriver parameters
service = webdriver.FirefoxService(service_args=['--log', 'debug'])
firefox_options = FirefoxOptions()
firefox_options.add_argument("--headless")
driver = webdriver.Firefox(service=service, options=firefox_options)

# -- Start LOAN APPLICATIONS script --

# Store the current window handle
main_window = driver.current_window_handle

# Open the login page
driver.get('https://backupfs.provisocloud.com:255/default.aspx')

# enter username
driver.find_element(By.ID, "txtUserID").send_keys("505")
# enter password
driver.find_element(By.ID, "txtUserPassword").send_keys("123456")
# click login
driver.find_element(By.ID, "cmbLogin").click()

# TEMP FIX TO OVERRIDE FFOX WARNINGS
time.sleep(10)
print("Login successful")

# Switch to popup window
for handle in driver.window_handles:
    if handle != main_window:
        popup = handle
        driver.switch_to.window(popup)

# click report menu
driver.find_element(By.ID, "TreeView15t0").click()
# click loans report
driver.find_element(By.ID, "TreeView15t46").click()
# click loan application report
driver.find_element(By.ID, "TreeView15t47").click()
# click loan application report
driver.find_element(By.ID, "TreeView15t48").click()

time.sleep(5)

# Switch to new popup window
for handle in driver.window_handles:
    if handle != main_window:
        popup = handle
        driver.switch_to.window(popup)

# DEFAULT TO TODAY'S DATE
# Run at 11pm every day
# from date
driver.find_element(By.ID, "txtDf").clear()
driver.find_element(By.ID, "txtDf").send_keys(datetime.datetime.today().strftime('%m/%d/%Y'))
# to date
driver.find_element(By.ID, "txtDt").clear()
driver.find_element(By.ID, "txtDt").send_keys(datetime.datetime.today().strftime('%m/%d/%Y'))

# click show report
button1 = WebDriverWait(driver, 600).until(
        EC.presence_of_element_located((By.ID, "Button1"))
    )
button1.click()

# click download report in Excel
button2 = WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.ID, "Button2"))
    )
button2.click()

print('Downloading file...')

time.sleep(60)

download_path = '/root/Downloads'
downloaded_files = os.listdir(download_path)
print(downloaded_files)

print("download report successful")

# -- End LOAN APPLICATIONS script --


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Login successful
['LOAN APPLICATION REPORT-FROM-07292024-TO-07292024-224.xls', 'LOAN APPLICATION REPORT-FROM-07292024-TO-07292024-155.xls', 'LOAN APPLICATION REPORT-FROM-07292024-TO-07292024-189.xls']
download report successful


In [20]:
# Import libraries
import pymssql

# Append loan application data
# Find the file with the specific pattern in its name
pattern = '*LOAN APPLICATION REPORT*.xls'
matching_files = fnmatch.filter(downloaded_files, pattern)

if matching_files:
    file_path = os.path.join(download_path, matching_files[0])
    print(f"Selected file: {file_path}")
else:
    print("No matching file found")

# Connect to the database
server = "gendercredit-server.database.windows.net"
database = 'gendercredit-DB'
user = 'sqladmin'
password = 'wuskik-kicCok-sigxo1'

conn = pymssql.connect(server, user, password, database)
cursor = conn.cursor()

print('Connected to Azure SQL DB')



df = pd.read_table(file_path)
df = clean_names(df)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.fillna(value=0)
df.columns = df.columns.str.replace('#', 'number')

print('Successfully read .xls file')

for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO loan_applications (
            docket_number, full_name, track_number, request_amt, disb_mode,
            contact_number, momo_number, bank_name, b_a_number, product_name, location,
            userid, created_by, date, time
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            row['docket_number'], row['full_name'], row['track_number'], row['request_amt'], row['disb_mode'],
            row['contact_number'], row['momo_number'], row['bank_name'], row['b_a_number'], row['product_name'], row['location'],
            row['userid'], row['created_by'], row['date'], row['time']
        )
    )

conn.commit()
cursor.close()
conn.close()

print("Successfully inserted data to SQL")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Selected file: /root/Downloads/LOAN APPLICATION REPORT-FROM-07292024-TO-07292024-224.xls
Connected to Azure SQL DB
Successfully read .xls file
Successfully inserted data to SQL
